In [7]:
import numpy as np
import pandas as pd

def simulate_heston_one_day(mu=0.05,
                            kappa=5.0,
                            theta=0.04,
                            xi=0.5,
                            rho=-0.5,
                            v0=0.04,
                            dt=None):
    """
    Simulate a single day's log-price X, variance v, and volatility sigma
    under Heston (1993) at 1-second frequency.
    """
    seconds_per_day = int(6.5 * 3600)             # 23,400 seconds
    if dt is None:
        dt = (1/252) / seconds_per_day            # time‐step in years

    N = seconds_per_day + 1
    X = np.empty(N)
    v = np.empty(N)
    sigma = np.empty(N)

    X[0] = np.log(100.0)
    v[0] = v0
    sigma[0] = np.sqrt(v0)

    cov = np.array([[1, rho], [rho, 1]])
    L = np.linalg.cholesky(cov)

    for t in range(1, N):
        z = np.random.randn(2)
        dW1, dW2 = (L @ z) * np.sqrt(dt)
        v_prev = v[t-1]
        dv = kappa * (theta - v_prev) * dt + xi * np.sqrt(max(v_prev,0)) * dW2
        v[t] = max(v_prev + dv, 0)
        sigma[t] = np.sqrt(v[t])
        X[t] = X[t-1] + (mu - 0.5 * v_prev)*dt + sigma[t-1]*dW1

    return X, v, sigma

def add_micro_noise(X, noise_sd=0.0005):
    return X + np.random.normal(0, noise_sd, size=X.shape)

def generate_time_varying_noise(variances, rho_eps):
    """
    Generate a zero-mean Gaussian noise series {eps[t]} with
    Var(eps[t]) = variances[t] and Corr(eps[t], eps[t-1]) = rho_eps.
    We use an AR(1)-style construction on standardized normals:
      u[t] = rho_eps * u[t-1] + sqrt(1 - rho_eps^2) * z[t], z[t] ~ N(0,1)
      eps[t] = sqrt(variances[t]) * u[t].
    """
    N = variances.shape[0]
    u = np.zeros(N)
    eps = np.zeros(N)
    # Initialize u[0] ~ N(0,1)
    u[0] = np.random.randn()
    eps[0] = np.sqrt(variances[0]) * u[0]
    # Iterate
    for t in range(1, N):
        z = np.random.randn()
        u[t] = rho_eps * u[t-1] + np.sqrt(1 - rho_eps**2) * z
        eps[t] = np.sqrt(variances[t]) * u[t]
    return eps

def subsampled_rv(Y, K):
    """
    Return:
      - the average of K subsampled realized variances
      - the list of sample‐sizes (n_k) for each of the K grids
    """
    subs = [np.sum(np.diff(Y[k::K])**2) for k in range(K)]
    counts = [len(Y[k::K]) - 1 for k in range(K)]
    return np.mean(subs), counts

def naively_sampled_rv(Y):
    return np.sum(np.diff(Y)**2)

def zma_rv(Y, K):
    """
    Compute the original ZMA:
      ZMA = max{ (n * avg_sub - bar_n * full) / (n - bar_n), 0 }.
    Returns the ZMA estimate.
    """
    n = len(Y) - 1
    full = naively_sampled_rv(Y)
    subsrv, counts = subsampled_rv(Y, K)
    avg_sub = subsrv
    bar_n = np.mean(counts)
    z = (n * avg_sub - bar_n * full) / (n - bar_n)
    return max(z, 0.0)

def monte_carlo_heston(M=1000,
                                 mu=0.05, kappa=5.0, theta=0.04,
                                 xi=0.5, rho=-0.5, noise_sd=0.0005):
    seconds = int(6.5*3600)
    dt = (1/252)/seconds
    T = 1/252
    var_eps = noise_sd**2

    results = {
        'Naive':    np.empty(M),
        'Subsample':np.empty(M),
        'ZMA':      np.empty(M),
        'FiveMin':  np.empty(M),
        'TrueIV':   np.empty(M),
        'K_sub':    np.empty(M),
        'K_zma':    np.empty(M)
    }

    for i in range(M):
        X, v, sigma = simulate_heston_one_day(mu=mu, kappa=kappa,
                                              theta=theta, xi=xi,
                                              rho=rho, v0=theta, dt=dt)
        Y = add_micro_noise(X, noise_sd)

        iv = np.sum(v[:-1]) * dt
        iq = np.sum(v[:-1]**2) * dt

        bar_n_star = ((T/(6*var_eps**2)) * iq)**(1/3)
        K_sub = max(1, int(round(seconds / bar_n_star)))

        eta2 = (4/3) * iq
        c_star = ((16 * var_eps**2) / (T * eta2))**(1/3)
        K_zma = max(1, int(round(c_star * seconds**(2/3))))

        results['Naive'][i]     = naively_sampled_rv(Y)
        results['Subsample'][i] = subsampled_rv(Y, K_sub)[0]
        results['ZMA'][i]       = zma_rv(Y, K_zma)

        idx_5min = np.arange(0, seconds + 1, 300) 
        Y_5min = Y[idx_5min]
        results['FiveMin'][i]   = np.sum(np.diff(Y_5min)**2)

        results['TrueIV'][i]    = iv
        results['K_sub'][i]     = K_sub
        results['K_zma'][i]     = K_zma

    return pd.DataFrame(results)

df = monte_carlo_heston(M=1000)

In [8]:
def compute_metrics(df):
    methods = ['Naive', 'Subsample', 'FiveMin', 'ZMA']
    rows = []
    for m in methods:
        diff = df[m] - df['TrueIV']
        bias     = diff.mean()
        variance = diff.var(ddof=0)      
        rmse     = np.sqrt((diff**2).mean())
        rows.append({
            'Method':   m,
            'Bias':     bias,
            'Variance': variance,
            'RMSE':     rmse
        })
    return pd.DataFrame(rows)

metrics_df = compute_metrics(df)
print(metrics_df)

      Method          Bias      Variance      RMSE
0      Naive  1.169730e-02  1.736962e-08  0.011698
1  Subsample  1.556131e-05  8.145025e-10  0.000033
2    FiveMin  3.727108e-05  1.070758e-09  0.000050
3        ZMA -3.540409e-07  7.919795e-11  0.000009


In [4]:
def monte_carlo_heston_noise_variations(
    M=500,
    mu=0.05, kappa=5.0, theta=0.04,
    xi=0.5, rho=-0.5,
    rho_eps=0.0,
    base_noise_var=0.0005**2,
    noise_var_amp=0.0
):
    """
    Monte Carlo over M days for different noise scenarios:
    - If noise_var_amp > 0, noise variance oscillates sinusoidally.
    - If rho_eps != 0, noise is AR(1) with serial correlation.
    Otherwise noise is i.i.d. homoskedastic.
    """
    seconds = int(6.5 * 3600)
    dt = (1/252) / seconds
    T = 1 / 252

    results = {
        'Naive':     np.empty(M),
        'Subsample': np.empty(M),
        'FiveMin':   np.empty(M),
        'ZMA':       np.empty(M),
        'TrueIV':    np.empty(M),
    }

    for i in range(M):
        X, v, sigma = simulate_heston_one_day(mu=mu, kappa=kappa, theta=theta,
                                              xi=xi, rho=rho, v0=theta, dt=dt)

        N = X.shape[0]
        t_idx = np.arange(N)
        variances = base_noise_var * (1 + noise_var_amp * np.sin(2 * np.pi * t_idx / N))

        eps = generate_time_varying_noise(variances, rho_eps)

        Y = X + eps

        iv = np.sum(v[:-1] * dt)

        avg_var_eps = np.mean(variances)
        iq = np.sum((v[:-1]**2) * dt)
        bar_n_star = ((T / (6 * avg_var_eps**2)) * iq)**(1/3)
        K_sub = max(1, int(round(seconds / bar_n_star)))

        eta2 = (4/3) * iq
        c_star = ((16 * avg_var_eps**2) / (T * eta2))**(1/3)
        K_zma = max(1, int(round(c_star * (seconds**(2/3)))))

        results['Naive'][i]     = naively_sampled_rv(Y)
        results['Subsample'][i] = subsampled_rv(Y, K_sub)[0]
        idx_5min = np.arange(0, seconds+1, 300)
        results['FiveMin'][i]   = np.sum(np.diff(Y[idx_5min])**2)
        results['ZMA'][i]       = zma_rv(Y, K_zma)
        results['TrueIV'][i]    = iv

    return pd.DataFrame(results)

df_hetero = monte_carlo_heston_noise_variations(
    M=1000, rho_eps=0.0, noise_var_amp=0.5
)

df_serial = monte_carlo_heston_noise_variations(
    M=1000, rho_eps=-0.5, noise_var_amp=0.0
)

df_both = monte_carlo_heston_noise_variations(
    M=1000, rho_eps=-0.5, noise_var_amp=0.5
)

print("Heteroskedastic, Uncorrelated Noise:")
print(df_hetero.head(), "\n")
print("Homoskedastic, Serially Correlated Noise:")
print(df_serial.head(), "\n")
print("Heteroskedastic & Serially Correlated Noise:")
print(df_both.head())

Heteroskedastic, Uncorrelated Noise:
      Naive  Subsample   FiveMin       ZMA    TrueIV
0  0.011859   0.000302  0.000252  0.000194  0.000184
1  0.011861   0.000173  0.000167  0.000141  0.000149
2  0.011818   0.000209  0.000187  0.000138  0.000144
3  0.011885   0.000226  0.000258  0.000175  0.000174
4  0.012172   0.000193  0.000222  0.000200  0.000197 

Homoskedastic, Serially Correlated Noise:
      Naive  Subsample   FiveMin  ZMA    TrueIV
0  0.018010   0.000181  0.000210  0.0  0.000163
1  0.017372   0.000159  0.000191  0.0  0.000150
2  0.017859   0.000171  0.000168  0.0  0.000155
3  0.017275   0.000115  0.000167  0.0  0.000131
4  0.017442   0.000212  0.000205  0.0  0.000163 

Heteroskedastic & Serially Correlated Noise:
      Naive  Subsample   FiveMin  ZMA    TrueIV
0  0.017780   0.000214  0.000183  0.0  0.000168
1  0.017805   0.000154  0.000165  0.0  0.000165
2  0.018396   0.000145  0.000158  0.0  0.000160
3  0.017933   0.000224  0.000293  0.0  0.000195
4  0.017879   0.000180  0.

In [ ]:
metrics_hetero = compute_metrics(df_hetero)
metrics_serial = compute_metrics(df_serial)
metrics_both   = compute_metrics(df_both)

metrics_hetero['Scenario'] = 'Heteroskedastic'
metrics_serial['Scenario'] = 'Serially Correlated'
metrics_both['Scenario']   = 'Both'

metrics_all = pd.concat([metrics_hetero, metrics_serial, metrics_both], ignore_index=True)

print(metrics_all)

       Method          Bias      Variance      RMSE             Scenario
0       Naive  1.169529e-02  2.021994e-08  0.011696      Heteroskedastic
1   Subsample  1.681428e-05  8.439256e-10  0.000034      Heteroskedastic
2     FiveMin  4.119314e-05  1.021049e-09  0.000052      Heteroskedastic
3         ZMA  1.380602e-07  8.048659e-11  0.000009      Heteroskedastic
4       Naive  1.753229e-02  6.701163e-08  0.017534  Serially Correlated
5   Subsample  1.486594e-05  7.814220e-10  0.000032  Serially Correlated
6     FiveMin  3.938688e-05  1.111776e-09  0.000052  Serially Correlated
7         ZMA -1.585861e-04  1.982600e-10  0.000159  Serially Correlated
8       Naive  1.755344e-02  7.297553e-08  0.017556                 Both
9   Subsample  1.666656e-05  7.719333e-10  0.000032                 Both
10    FiveMin  3.871690e-05  1.082776e-09  0.000051                 Both
11        ZMA -1.595753e-04  2.103814e-10  0.000160                 Both


In [ ]:
def simulate_heston_bates(
    mu=0.05,
    kappa=5.0,
    theta=0.04,
    xi=0.5,
    rho=-0.5,
    v0=0.04,
    T=1/252,
    seconds_per_day=int(6.5 * 3600),
    jump_intensity=1260,
    jump_mu=0.0,
    jump_sigma=0.02
):
    """
    Simulate one trading day under the Heston–Bates model (no intraday seasonality).
    - X: log-price
    - v: variance
    - J: jump multipliers (1 if no jump, lognormal if jump)
    - dt: time step
    """
    dt = T / seconds_per_day
    X = np.empty(seconds_per_day + 1)
    v = np.empty(seconds_per_day + 1)
    J = np.ones(seconds_per_day + 1)

    X[0] = np.log(100.0)
    v[0] = v0

    L = np.linalg.cholesky([[1, rho], [rho, 1]])

    for i in range(1, seconds_per_day + 1):
        z = np.random.randn(2)
        dW1, dW2 = (L @ z) * np.sqrt(dt)

        dv = kappa * (theta - v[i-1]) * dt + xi * np.sqrt(max(v[i-1], 0)) * dW2
        v[i] = max(v[i-1] + dv, 0.0)

        if np.random.rand() < jump_intensity * dt:
            J[i] = np.random.lognormal(mean=jump_mu, sigma=jump_sigma)
        else:
            J[i] = 1.0


        X[i] = (
            X[i-1]
            + (mu - 0.5 * v[i-1]) * dt
            + np.sqrt(v[i-1]) * dW1
            + np.log(J[i])
        )

    return X, v, J, dt

def monte_carlo_bates(
    M=500,
    noise_sd=0.0005,
    mu=0.05,
    kappa=5.0,
    theta=0.04,
    xi=0.5,
    rho=-0.5,
    v0=0.04,
    T=1/252,
    seconds_per_day=int(6.5 * 3600),
    jump_intensity=300,
    jump_mu=0.0,
    jump_sigma=0.02
):
    """
    Monte Carlo over M days for Heston–Bates without U-shape seasonality.
    For each day:
      1) Simulate Heston–Bates log-prices X, variances v, jumps J.
      2) Add microstructure noise to X → Y.
      3) Compute true quadratic variation: ∫_0^T v_t dt + sum_jumps (log J)^2.
      4) Compute integrated quarticity: ∫_0^T v_t^2 dt.
      5) Determine optimal subsampling K_sub and ZMA K_zma.
      6) Compute:
         - Naive RV (1-second)
         - Subsampled RV
         - 5-minute RV
         - ZMA RV
    Returns a DataFrame with columns: Naive, Subsample, FiveMin, ZMA, TrueQV.
    """
    var_eps = noise_sd**2
    out = {
        'Naive':     np.empty(M),
        'Subsample': np.empty(M),
        'FiveMin':   np.empty(M),
        'ZMA':       np.empty(M),
        'TrueQV':    np.empty(M),
        'K_sub':     np.empty(M),
        'K_zma':     np.empty(M)
    }

    for i in range(M):
        X, v, J, dt = simulate_heston_bates(
            mu=mu, kappa=kappa, theta=theta, xi=xi,
            rho=rho, v0=v0, T=T, seconds_per_day=seconds_per_day,
            jump_intensity=jump_intensity, jump_mu=jump_mu, jump_sigma=jump_sigma
        )

        Y = add_micro_noise(X, noise_sd)

        cont_var = np.sum(v[:-1]) * dt

        jump_var = np.sum(np.log(J)**2)

        true_qv = cont_var + jump_var
        out['TrueQV'][i] = true_qv

        iq = np.sum(v[:-1]**2) * dt

        bar_n_star = ((T / (6 * var_eps**2)) * iq)**(1/3)
        K_sub = max(1, int(round(seconds_per_day / bar_n_star)))
        out['K_sub'][i] = K_sub

        eta2 = (4/3) * iq
        c_star = ((16 * var_eps**2) / (T * eta2))**(1/3)
        K_zma = max(1, int(round(c_star * (seconds_per_day**(2/3)))))
        out['K_zma'][i] = K_zma

        out['Naive'][i] = naively_sampled_rv(Y)

        subs_rv, _ = subsampled_rv(Y, K_sub)
        out['Subsample'][i] = subs_rv

        idx_5min = np.arange(0, seconds_per_day + 1, 300)
        Y_5min = Y[idx_5min]
        out['FiveMin'][i] = np.sum(np.diff(Y_5min)**2)

        out['ZMA'][i] = zma_rv(Y, K_zma)

    return pd.DataFrame(out)

df = monte_carlo_bates(
    M=1000,
    noise_sd=0.0005,
    mu=0.05, kappa=5.0, theta=0.04, xi=0.5, rho=-0.5, v0=0.04,
    jump_intensity=300, jump_mu=0.0, jump_sigma=0.02
)

print(df.describe())

            Naive   Subsample     FiveMin         ZMA      TrueQV       K_sub  \
count 1.00000e+03 1.00000e+03 1.00000e+03 1.00000e+03 1.00000e+03 1.00000e+03   
mean  1.23433e-02 6.42948e-04 6.82377e-04 6.39856e-04 6.40067e-04 5.79267e+02   
std   7.76731e-04 7.58545e-04 7.74844e-04 7.66545e-04 7.65662e-04 3.41897e+01   
min   1.14913e-02 9.67219e-05 1.16751e-04 1.07587e-04 1.12759e-04 4.89000e+02   
25%   1.18669e-02 1.85240e-04 2.07766e-04 1.66619e-04 1.66955e-04 5.55000e+02   
50%   1.20447e-02 2.99407e-04 3.36323e-04 2.91044e-04 2.86134e-04 5.78000e+02   
75%   1.25329e-02 7.70859e-04 8.13340e-04 7.86870e-04 7.85777e-04 6.01000e+02   
max   1.70444e-02 5.54294e-03 5.50729e-03 5.66832e-03 5.64889e-03 7.16000e+02   

            K_zma  
count 1.00000e+03  
mean  2.55190e+01  
std   1.54078e+00  
min   2.20000e+01  
25%   2.40000e+01  
50%   2.50000e+01  
75%   2.60000e+01  
max   3.20000e+01  


In [199]:
sp500 = pd.read_csv('USA500IDXUSD.csv')
sp500

,Date,Timestamp,Open,High,Low,Close,Volume
0,20250202,23:00:00,5949.989,5949.989,5945.498,5945.498,0.00469
1,20250202,23:00:01,5942.751,5944.492,5942.504,5943.754,0.00737
2,20250202,23:00:02,5942.242,5942.501,5937.501,5937.501,0.00938
3,20250202,23:00:03,5936.236,5936.236,5934.001,5934.245,0.00670
4,20250202,23:00:04,5934.179,5934.179,5927.989,5927.989,0.00804
...,...,...,...,...,...,...,...
3384376,20250525,23:59:51,5851.634,5851.634,5851.498,5851.498,0.00122
3384377,20250525,23:59:55,5851.245,5851.245,5851.245,5851.245,0.00061
3384378,20250525,23:59:57,5851.742,5851.742,5851.742,5851.742,0.00061
3384379,20250525,23:59:58,5851.625,5851.754,5851.625,5851.754,0.00122


In [ ]:
def prepare_data(df_orig):
    df = df_orig.copy()
    df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Timestamp'])
    df.set_index('Datetime', inplace=True)
    df['log_close'] = np.log(df['Close'])
    return df

def realized_quadpower_quarticity(log_prices):
    returns = np.diff(log_prices)
    M = len(returns)
    prod4 = (
        pd.Series(np.abs(returns))
          .rolling(window=4)
          .apply(np.product, raw=True)
          .dropna()
          .values
    )
    return (M * (np.pi**2) / 4) * prod4.sum()

def daily_rv_estimators(df):
    df = df.copy()
    T = 1/252
    results = []
    for date, group in df.groupby(df.index.date):
        lp = group['log_close'].values
        n = len(lp) - 1
        if n < 1:
            continue

        full_rv = naively_sampled_rv(lp)
        var_eps_hat = full_rv / (2 * n)

        RQQ = realized_quadpower_quarticity(lp)

        if var_eps_hat > 0 and RQQ > 0:
            bar_n_star = ((T / (6 * var_eps_hat**2)) * RQQ)**(1/3)
            K_boost = max(1, int(round(n / bar_n_star)))
        else:
            K_boost = 1

        if var_eps_hat > 0 and RQQ > 0:
            eta2 = (4/3) * RQQ
            c_star = ((16 * var_eps_hat**2) / (T * eta2))**(1/3)
            K_zma = max(1, int(round(c_star * n**(2/3))))
        else:
            K_zma = 1

        rv_naive = full_rv

        rv_boost = subsampled_rv(lp, K_boost)

        rv_zma = zma_rv(lp, K_zma)

        idx_5min = np.arange(0, len(lp), 300)
        lp_5min = lp[idx_5min]
        rv_5min = naively_sampled_rv(lp_5min)

        results.append({
            'date':      date,
            'var_eps':   var_eps_hat,
            'RQQ':       RQQ,
            'K_boost':   K_boost,
            'K_zma':     K_zma,
            'RV_naive':  rv_naive,
            'RV_5min':   rv_5min,
            'RV_boost':  rv_boost,
            'RV_ZMA':    rv_zma
        })

    return pd.DataFrame(results).set_index('date')

In [201]:
df_loaded_1year = pd.read_csv('1 year.csv')
df_prepared_1year = prepare_data(df_loaded_1year)

In [202]:
daily_metrics_1year = daily_rv_estimators(df_prepared_1year)
print(daily_metrics_1year)

                 var_eps           RQQ  K_boost  K_zma      RV_naive  \
date                                                                   
2024-05-26  7.837688e-10  1.502585e-12       48      7  8.872263e-07   
2024-05-27  6.560155e-10  1.233080e-10      109      7  8.230370e-06   
2024-05-28  1.076464e-09  1.925602e-09      165      8  3.675480e-05   
2024-05-29  1.038862e-09  3.625128e-09      166      7  4.494947e-05   
2024-05-30  1.198683e-09  4.049843e-09      168      8  4.961109e-05   
...                  ...           ...      ...    ...           ...   
2025-05-20  9.220182e-10  5.330443e-09      209      8  6.203338e-05   
2025-05-21  1.544324e-09  3.177101e-08      194      7  1.237189e-04   
2025-05-22  1.383416e-09  1.577374e-08      227      8  1.107120e-04   
2025-05-23  2.492150e-09  8.562416e-08      198      7  2.060908e-04   
2025-05-25  2.836448e-09  1.407859e-09       83      7  2.298658e-05   

                 RV_5min      RV_boost        RV_ZMA  
date    

In [ ]:
daily_log_1year = df_prepared_1year['log_close'].resample('D').last().rename('log_close')
daily_log_1year.index = daily_log_1year.index.date
combined_1year = daily_metrics_1year.merge(
    daily_log_1year,
    left_index=True,
    right_index=True,
    how='left'
)

In [205]:
combined_1year.to_csv('daily_rv_estimators_1year_with_log3.csv', index=True)